In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
prev_path = "\\".join(list(os.getcwd().split('\\')[0:-1]))
if module_path not in sys.path: sys.path.append(prev_path)

# you will need to pip install eikon=1.1.15 and maybe some other packages
# it's best if we share the venv 

from models.data.source import Eikon
from models.stat_models.linearregression import LinearRegressionModel, MultiOutputLinearRegressionModel
from models.unsupervised.pca import PcaHandler
from models.data.handler import DataHandler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data_path = r'C:\Users\serge\IdeaProjects\portfolio_manager\data\fama-french-factors'
path_apikeys = r'C:\Users\serge\OneDrive\Documents\apikeys.csv'

In [3]:
rics_list = [
    'LVMH.PA', 'ASML.AS', 'NESN.S', 'LIN',
    'NOVOb.CO', 'AAPL.O', 'ROG.S', 'UNH',
    'SAPG.DE', 'MSFT.O'
]

params = {
    'rics': rics_list,
    'load_path': r'C:\Users\serge\IdeaProjects\portfolio_manager\portfolio_management\models\data\csv',
    'field': ['TR.PriceClose', 'Price Close'],
    'date_field': ['TR.PriceClose.calcdate', 'Calc Date'],
}

In [4]:
N_PCA_COMPONENTS = 5
HORIZON_IN_DAYS = 14
RIC = "ASML.AS"

In [5]:
eikon_api = Eikon(path_apikeys)

2023-06-19 01:07:51,131 P[5824] [MainThread 16652] Error: no proxy address identified.
Check if Eikon Desktop or Eikon API Proxy is running.
2023-06-19 01:07:51,138 P[5824] [MainThread 16652] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2023-06-19 01:07:51,140 P[5824] [MainThread 16652] Error on handshake url http://127.0.0.1:None/api/handshake : UnsupportedProtocol("Request URL is missing an 'http://' or 'https://' protocol.")
2023-06-19 01:07:51,144 P[5824] [MainThread 16652] Port number was not identified, cannot send any request


In [6]:
 eikon_api.load_timeseries(**params)

Loaded: LVMH.PA
Loaded: ASML.AS
Loaded: NESN.S
Loaded: LIN
Loaded: NOVOb.CO
Loaded: AAPL.O
Loaded: ROG.S
Loaded: UNH
Loaded: SAPG.DE
[Errno 2] No such file or directory: 'C:\\Users\\serge\\IdeaProjects\\portfolio_manager\\portfolio_management\\models\\data\\csv\\MSFT.O.csv'


In [7]:
data = eikon_api.data.copy()
data.head(20)

,Calc Date,LVMH.PA,ASML.AS,NESN.S,LIN,NOVOb.CO,AAPL.O,ROG.S,UNH,SAPG.DE
0,2000-01-04,72.458651,38.039234,28.00,24.46875,17.550,0.915178,175.423964,6.632812,39.076938
1,2000-01-05,70.007313,34.411467,27.80,24.90625,17.136,0.928570,171.065003,6.617188,34.361184
2,2000-01-06,69.214234,32.004583,28.96,26.75000,17.550,0.848213,172.808587,6.859375,35.475892
3,2000-01-07,69.214234,34.394026,29.53,26.78125,17.910,0.888392,182.592033,7.664062,43.013639
4,2000-01-10,72.134209,36.870674,29.23,26.81250,19.260,0.872767,184.819946,7.531250,44.770651
5,2000-01-11,73.540123,37.324145,29.30,27.25000,19.170,0.828124,183.076362,7.546875,46.204439
6,2000-01-12,76.784540,36.800910,29.93,26.78125,18.990,0.778459,183.076362,7.476562,47.157535
7,2000-01-13,81.470921,37.202057,30.05,26.40625,18.900,0.863838,183.076362,7.710938,47.654802
8,2000-01-14,84.715338,41.597237,30.21,26.71875,18.360,0.896762,184.045020,7.703125,48.732215
9,2000-01-17,83.814111,43.602974,29.95,26.71875,18.360,0.896762,184.626215,7.703125,47.986314


In [8]:
data_handler = DataHandler(data=data, date_col=params['date_field'][1])
df = data_handler.get_returns(period=HORIZON_IN_DAYS, out=True)

TypeError: loop of ufunc does not support argument 0 of type numpy.float64 which has no callable log method

In [ ]:
# T = len(data)
# tau = 12
# df = {}
# for t in np.arange(0, T-tau, tau):
#     df[data.iloc[t+tau,0]]= np.log(data.iloc[t+tau,1:].astype(float).values / data.iloc[t,1:].astype(float).values)
# df = pd.DataFrame.from_dict(df, orient='index')
# df.columns = data.columns[1:]

# this is unit test for tau=12
# assert np.isclose(df.iloc[0,0],np.log(79.848712 / 72.458651))

In [ ]:
df.head()

In [ ]:
pca_model = PcaHandler(np.array(df))

In [ ]:
pca_model.plot(N_PCA_COMPONENTS)

In [ ]:
pca_factors = pca_model.components(N_PCA_COMPONENTS)
x = pca_factors

In [ ]:
pca_model.singular_values

In [ ]:
pca_model.benchmark_test()

In [ ]:
mod = LinearRegressionModel(x=x, y=np.array(df[RIC]))

In [ ]:
mod.plot()

In [ ]:
plt.plot(mod.residuals)

In [ ]:
sns.displot(mod.residuals)

In [ ]:
## Rolling estimation

In [ ]:
T

In [ ]:
T = len(df)
period = int(T/2)
params = {}
for i in range(T):
    try:
        df_sliced = df.iloc[i:i+period, :]
        pca_model = PcaHandler(df_sliced)
        factors = pca_model.components(N_PCA_COMPONENTS)
        mod = LinearRegressionModel(x=factors, y=np.array(df_sliced["ASML.AS"]))
        params[i] = mod.beta.flatten()
    except:
        params[i] = np.repeat(np.nan, N_PCA_COMPONENTS)

In [ ]:
rolling_betas = pd.DataFrame.from_dict(params, orient='index')

In [ ]:
rolling_betas

In [ ]:
fig, axs = plt.subplots(N_PCA_COMPONENTS,1,figsize=(11,8))
for i in range(N_PCA_COMPONENTS):
    rolling_betas.iloc[:,i].plot(ax=axs[i])
plt.show()

In [ ]:
rolling_betas.dropna()